# Reflectance SMOOTHING ANALYSIS

## Import modules

In [2]:
import array
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from termcolor import colored
import time
import xarray as xr

%reload_ext Cython

## Functions

Cython functions, modified `ws2dvopt` and `ws2dvoptp` in order to get vcurve

In [50]:
%%cython

from cpython.array cimport array, clone
from libc.math cimport log, pow, sqrt
cimport numpy as np
import numpy as np

tFloat = np.double
ctypedef np.double_t dtype_t


cpdef lag1corr(np.ndarray[dtype_t] data1, np.ndarray[dtype_t] data2, double nd):
    """Calculates Lag-1 autocorrelation.

    Adapted from https://stackoverflow.com/a/29194624/5997555

    Args:
        data1: fist data series
        data2: second data series
        nd: no-data value (will be exluded from calulation)

    Returns:
        Lag-1 autocorrelation value
    """

    cdef int M, sub
    cdef double sum1, sum2, var_sum1, var_sum2, cross_sum, std1, std2, cross_mean

    M = data1.size

    sum1 = 0.
    sum2 = 0.
    sub = 0
    for i in range(M):
        if data1[i] != nd and data2[i] != nd:
            sum1 += data1[i]
            sum2 += data2[i]
        else:
            sub += 1
    mean1 = sum1 / (M-sub)
    mean2 = sum2 / (M-sub)

    var_sum1 = 0.
    var_sum2 = 0.
    cross_sum = 0.
    for i in range(M):
        if data1[i] != nd and data2[i] != nd:
            var_sum1 += (data1[i] - mean1) ** 2
            var_sum2 += (data2[i] - mean2) ** 2
            cross_sum += (data1[i] * data2[i])

    std1 = (var_sum1 / (M-sub)) ** .5
    std2 = (var_sum2 / (M-sub)) ** .5
    cross_mean = cross_sum / (M-sub)
    return (cross_mean - mean1 * mean2) / (std1 * std2)

cpdef ws2d(np.ndarray[dtype_t] y, double lmda, np.ndarray[dtype_t] w):
    cdef array dbl_array_template = array('d', [])
    cdef int i, i1, i2, m, n
    cdef array z, d, c, e

    n = y.shape[0]
    m = n - 1

    z = clone(dbl_array_template, n, zero=False)
    d = clone(dbl_array_template, n, zero=False)
    c = clone(dbl_array_template, n, zero=False)
    e = clone(dbl_array_template, n, zero=False)

    d.data.as_doubles[0] = w[0] + lmda
    c.data.as_doubles[0] = (-2 * lmda) / d.data.as_doubles[0]
    e.data.as_doubles[0] = lmda /d.data.as_doubles[0]
    z.data.as_doubles[0] = w[0] * y[0]
    d.data.as_doubles[1] = w[1] + 5 * lmda - d.data.as_doubles[0] * (c.data.as_doubles[0] * c.data.as_doubles[0])
    c.data.as_doubles[1] = (-4 * lmda - d.data.as_doubles[0] * c.data.as_doubles[0] * e.data.as_doubles[0]) / d.data.as_doubles[1]
    e.data.as_doubles[1] =  lmda / d.data.as_doubles[1]
    z.data.as_doubles[1] = w[1] * y[1] - c.data.as_doubles[0] * z.data.as_doubles[0]
    for i in range(2, m-1):
        i1 = i - 1
        i2 = i - 2
        d.data.as_doubles[i]= w[i] + 6 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
        c.data.as_doubles[i] = (-4 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1])/ d.data.as_doubles[i]
        e.data.as_doubles[i] =  lmda / d.data.as_doubles[i]
        z.data.as_doubles[i] = w[i] * y[i] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 2
    i2 = m - 3
    d.data.as_doubles[m - 1] = w[m - 1] + 5 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    c.data.as_doubles[m - 1] = (-2 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1]) / d.data.as_doubles[m - 1]
    z.data.as_doubles[m - 1] = w[m - 1] * y[m - 1] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 1
    i2 = m - 2
    d.data.as_doubles[m] = w[m] +  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    z.data.as_doubles[m] = (w[m] * y[m] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]) / d.data.as_doubles[m]
    z.data.as_doubles[m - 1] = z.data.as_doubles[m - 1] / d.data.as_doubles[m - 1] - c.data.as_doubles[m - 1] * z.data.as_doubles[m]
    for i in range(m-2, -1, -1):
        z.data.as_doubles[i] = z.data.as_doubles[i] / d.data.as_doubles[i] - c.data.as_doubles[i] * z.data.as_doubles[i + 1] - e.data.as_doubles[i] * z.data.as_doubles[i + 2]
    return z

cpdef ws2dp(np.ndarray[dtype_t] y, double lmda, np.ndarray[dtype_t] w, double p):
  """Whittaker smoother with asymmetric smoothing and fixed lambda (S).

  Args:
      y: time-series numpy array
      l: smoothing parameter lambda (S)
      w: weights numpy array
      p: "Envelope" value

  Returns:
      Smoothed time-series array z
  """
  cdef array template = array('d', [])
  cdef int m, i, j
  cdef double y_tmp, z_tmp, p1

  m = y.shape[0]
  i = 0
  j = 0
  p1 = 1-p

  template = array('d', [])
  z = clone(template, m, True)
  znew = clone(template, m, True)
  wa = clone(template, m, False)
  ww = clone(template, m, False)

  # Calculate weights

  for i in range(10):
    for j in range(m):
      y_tmp = y[j]
      z_tmp = z.data.as_doubles[j]

      if y_tmp > z_tmp:
        wa.data.as_doubles[j] = p
      else:
        wa.data.as_doubles[j] = p1
      ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

    znew[0:m] = _ws2d(y, lmda, ww)
    z_tmp = 0.0
    j = 0
    for j in range(m):
      z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

    if z_tmp == 0.0:
      break

    z[0:m]= znew[0:m]

  z[0:m] = _ws2d(y, lmda, ww)
  return z


cdef _ws2d(np.ndarray[dtype_t] y, double lmda, array[double] w):
    """Internal whittaker function for use in asymmetric smoothing.
    Args:
      y: time-series numpy array
      lmbda: lambda (s) value
      w: weights numpy array
    Returns:
        smoothed time-series array z
    """

    cdef array dbl_array_template = array('d', [])
    cdef int i, i1, i2, m, n
    cdef array z, d, c, e

    n = y.shape[0]
    m = n - 1

    z = clone(dbl_array_template, n, zero=False)
    d = clone(dbl_array_template, n, zero=False)
    c = clone(dbl_array_template, n, zero=False)
    e = clone(dbl_array_template, n, zero=False)

    d.data.as_doubles[0] = w.data.as_doubles[0] + lmda
    c.data.as_doubles[0] = (-2 * lmda) / d.data.as_doubles[0]
    e.data.as_doubles[0] = lmda /d.data.as_doubles[0]
    z.data.as_doubles[0] = w.data.as_doubles[0] * y[0]
    d.data.as_doubles[1] = w.data.as_doubles[1] + 5 * lmda - d.data.as_doubles[0] * (c.data.as_doubles[0] * c.data.as_doubles[0])
    c.data.as_doubles[1] = (-4 * lmda - d.data.as_doubles[0] * c.data.as_doubles[0] * e.data.as_doubles[0]) / d.data.as_doubles[1]
    e.data.as_doubles[1] =  lmda / d.data.as_doubles[1]
    z.data.as_doubles[1] = w.data.as_doubles[1] * y[1] - c.data.as_doubles[0] * z.data.as_doubles[0]
    for i in range(2, m-1):
        i1 = i - 1
        i2 = i - 2
        d.data.as_doubles[i]= w.data.as_doubles[i] + 6 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
        c.data.as_doubles[i] = (-4 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1])/ d.data.as_doubles[i]
        e.data.as_doubles[i] =  lmda / d.data.as_doubles[i]
        z.data.as_doubles[i] = w.data.as_doubles[i] * y[i] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 2
    i2 = m - 3
    d.data.as_doubles[m - 1] = w.data.as_doubles[m - 1] + 5 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    c.data.as_doubles[m - 1] = (-2 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1]) / d.data.as_doubles[m - 1]
    z.data.as_doubles[m - 1] = w.data.as_doubles[m - 1] * y[m - 1] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 1
    i2 = m - 2
    d.data.as_doubles[m] = w.data.as_doubles[m] +  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    z.data.as_doubles[m] = (w.data.as_doubles[m] * y[m] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]) / d.data.as_doubles[m]
    z.data.as_doubles[m - 1] = z.data.as_doubles[m - 1] / d.data.as_doubles[m - 1] - c.data.as_doubles[m - 1] * z.data.as_doubles[m]
    for i in range(m-2, -1, -1):
        z.data.as_doubles[i] = z.data.as_doubles[i] / d.data.as_doubles[i] - c.data.as_doubles[i] * z.data.as_doubles[i + 1] - e.data.as_doubles[i] * z.data.as_doubles[i + 2]
    return z


cpdef ws2doptv(np.ndarray[dtype_t] y, np.ndarray[dtype_t] w, array[double] llas):
    """Whittaker smoother with normal V-curve optimization of lambda (S).
    Args:
        y: time-series numpy array
        w: weights numpy array
        llas: array with lambda values to iterate (S-range)
    Returns:
        Smoothed time-series array z and optimized lambda (S) value lopt
    """
    cdef array template = array('d', [])
    cdef array fits, pens, diff1, lamids, v, z
    cdef int m, m1, m2, nl, nl1, lix, i, k
    cdef double w_tmp, y_tmp, z_tmp, z2, llastep, f1, f2, p1, p2, l, l1, l2, vmin, lopt

    m = y.shape[0]
    m1 = m - 1
    m2 = m - 2
    nl = len(llas)
    nl1 = nl - 1
    i = 0
    k = 0

    template = array('d', [])

    fits = clone(template, nl, True)
    pens = clone(template, nl, True)
    z = clone(template, m, False)
    diff1 = clone(template, m1, True)
    lamids = clone(template, nl1, False)
    v = clone(template, nl1, False)

    # Compute v-curve
    for lix in range(nl):
        l = pow(10,llas.data.as_doubles[lix])
        z[0:m] = ws2d(y, l, w)
        for i in range(m):
            w_tmp = w[i]
            y_tmp = y[i]
            z_tmp = z.data.as_doubles[i]
            fits.data.as_doubles[lix] += pow(w_tmp * (y_tmp - z_tmp),2) 
        fits.data.as_doubles[lix] = log(fits.data.as_doubles[lix])

        for i in range(m1):
            z_tmp = z.data.as_doubles[i]
            z2 = z.data.as_doubles[i+1]
            diff1.data.as_doubles[i] = z2 - z_tmp
        for i in range(m2):
            z_tmp = diff1.data.as_doubles[i]
            z2 = diff1.data.as_doubles[i+1]
            pens.data.as_doubles[lix] += pow(z2 - z_tmp,2)
        pens.data.as_doubles[lix] = log(pens.data.as_doubles[lix])

    # Construct v-curve
    llastep = llas[1] - llas[0]

    for i in range(nl1):
        l1 = llas.data.as_doubles[i]
        l2 = llas.data.as_doubles[i+1]
        f1 = fits.data.as_doubles[i]
        f2 = fits.data.as_doubles[i+1]
        p1 = pens.data.as_doubles[i]
        p2 = pens.data.as_doubles[i+1]
        v.data.as_doubles[i] = sqrt(pow(f2 - f1,2) + pow(p2 - p1,2)) / (log(10) * llastep)
        lamids.data.as_doubles[i] = (l1+l2) / 2

    vmin = v.data.as_doubles[k]
    for i in range(1, nl1):
        if v.data.as_doubles[i] < vmin:
            vmin = v.data.as_doubles[i]
            k = i

    lopt = pow(10, lamids.data.as_doubles[k])

    z[0:m] = ws2d(y, lopt, w)

    return z, lopt, v, lamids



cpdef ws2doptvp(np.ndarray[dtype_t] y, np.ndarray[dtype_t] w, array[double] llas, double p):
    """Whittaker smoother with asymmetric V-curve optimization of lambda (S).
    Args:
        y: time-series numpy array
        w: weights numpy array
        llas: array with lambda values to iterate (S-range)
        p: "Envelope" value
    Returns:
        Smoothed time-series array z and optimized lambda (S) value lopt
    """
    cdef array template = array('d', [])
    cdef array fits, pens, diff1, lamids, v, z
    cdef int m, m1, m2, nl, nl1, lix, i, j, k
    cdef double w_tmp, y_tmp, z_tmp, z2, llastep, fit1, fit2, pen1, pen2, l, l1, l2, vmin, lopt, p1

    m = y.shape[0]
    m1 = m - 1
    m2 = m - 2
    nl = len(llas)
    nl1 = nl - 1
    i = 0
    k = 0
    j = 0
    p1 = 1-p

    template = array('d', [])
    fits = clone(template, nl, True)
    pens = clone(template, nl, True)
    z = clone(template, m, True)
    znew = clone(template, m, True)
    diff1 = clone(template, m1, True)
    lamids = clone(template, nl1, False)
    v = clone(template, nl1, False)
    wa = clone(template, m, False)
    ww = clone(template, m, False)

    # Compute v-curve
    for lix in range(nl):
        l = pow(10,llas.data.as_doubles[lix])

        for i in range(10):
          for j in range(m):
            y_tmp = y[j]
            z_tmp = z.data.as_doubles[j]
            if y_tmp > z_tmp:
              wa.data.as_doubles[j] = p
            else:
              wa.data.as_doubles[j] = p1
            ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

          znew[0:m] = _ws2d(y, l, ww)
          z_tmp = 0.0
          j = 0
          for j in range(m):
            z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

          if z_tmp == 0.0:
            break

          z[0:m]= znew[0:m]

        for i in range(m):
            w_tmp = w[i]
            y_tmp = y[i]
            z_tmp = z.data.as_doubles[i]
            fits.data.as_doubles[lix] += pow(w_tmp * (y_tmp - z_tmp),2)
        fits.data.as_doubles[lix] = log(fits.data.as_doubles[lix])

        for i in range(m1):
            z_tmp = z.data.as_doubles[i]
            z2 = z.data.as_doubles[i+1]
            diff1.data.as_doubles[i] = z2 - z_tmp
        for i in range(m2):
            z_tmp = diff1.data.as_doubles[i]
            z2 = diff1.data.as_doubles[i+1]
            pens.data.as_doubles[lix] += pow(z2 - z_tmp,2)
        pens.data.as_doubles[lix] = log(pens.data.as_doubles[lix])

    # Construct v-curve
    llastep = llas[1] - llas[0]

    for i in range(nl1):
        l1 = llas.data.as_doubles[i]
        l2 = llas.data.as_doubles[i+1]
        fit1 = fits.data.as_doubles[i]
        fit2 = fits.data.as_doubles[i+1]
        pen1 = pens.data.as_doubles[i]
        pen2 = pens.data.as_doubles[i+1]
        v.data.as_doubles[i] = sqrt(pow(fit2 - fit1,2) + pow(pen2 - pen1,2)) / (log(10) * llastep)
        lamids.data.as_doubles[i] = (l1+l2) / 2

    vmin = v.data.as_doubles[k]
    for i in range(1, nl1):
        if v.data.as_doubles[i] < vmin:
            vmin = v.data.as_doubles[i]
            k = i

    lopt = pow(10, lamids.data.as_doubles[k])

    del z
    z = clone(template, m, True)

    for i in range(10):
      for j in range(m):
        y_tmp = y[j]
        z_tmp = z.data.as_doubles[j]

        if y_tmp > z_tmp:
          wa.data.as_doubles[j] = p
        else:
          wa.data.as_doubles[j] = p1
        ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

      znew[0:m] = _ws2d(y, lopt, ww)
      z_tmp = 0.0
      j = 0
      for j in range(m):
        z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

      if z_tmp == 0.0:
        break

      z[0:m]= znew[0:m]

    z[0:m] = _ws2d(y, lopt, ww)
    
    return z, lopt, v, lamids

Other functions

In [4]:
def fromstring(x):
    
    '''Converts string to datetime object'''
    
    try:
        d = datetime.datetime.strptime(x, '%d/%m/%Y').date()
    except:
        d = datetime.datetime.strptime(x, '%Y-%m-%d').date()
        
    return d

def fromjulian(x):
    
    '''Converts julian to datetime object'''

    return datetime.datetime.strptime(x, '%Y%j').date()

Extract time series functions

In [5]:
def select_satellite(sat: str, ndvi_MD: tuple, ref_MD: tuple):
    
    (ndvi_MOD, ndvi_MYD, ndvi_MXD) = ndvi_MD
    (ref_MOD, ref_MYD, ref_MXD) = ref_MD
    
    if (sat == 'MYD'):
        ndvi_df = ndvi_MYD
        ref_df = ref_MYD
    elif (sat == 'MOD'):
        ndvi_df = ndvi_MOD
        ref_df = ref_MOD
    else:
        ndvi_df = ndvi_MXD
        ref_df = ref_MXD
        
    return (ndvi_df, ref_df)

def ndvi_extract_ts(df: pd.DataFrame, location: str, date_begin: datetime.date, date_end: datetime.date):
    
    y = df['NDVI'].loc[location].values

    dts = df['Date'].loc[location].values
    
    c_dts = df['Composite_date'].loc[location].values

    same_c = df['Same_composite'].loc[location].values
    
    #Crop to date range
    date_range = np.all([dts>=date_begin, dts<=date_end], axis=0)
    y = y[date_range]
    dts = dts[date_range]
    c_dts = c_dts[date_range]
    same_c = same_c[date_range]
    
    return (y, dts, c_dts, same_c)

def ref_extract_ts(df: pd.DataFrame, location: str, date_begin: datetime.date, date_end: datetime.date):
    
    y1 = df['B1'].loc[location].values
    y2 = df['B2'].loc[location].values

    dts = df['Date'].loc[location].values
    
    c_dts = df['Composite_date'].loc[location].values
    
    #Crop to date range
    date_range = np.all([dts>=date_begin, dts<=date_end], axis=0)
    y1 = y1[date_range]
    y2 = y2[date_range]
    dts = dts[date_range]
    c_dts = c_dts[date_range]
    
    return (y1, y2, dts, c_dts)

Derive NDVI

In [ ]:
def compute_ndvi_float(b1: float, b2:float):
    
    if ((b2 + b1) == 0):
        ndvi = 0
    else:
        ndvi = (b2 - b1)/(b2 + b1)
        
    return ndvi

In [14]:
def compute_ndvi_array(ref_b1: np.ndarray, ref_b2: np.ndarray):
    
    # derive NDVI
    ndvi = []
    for i in range(len(ref_b1)):
        ndvi.append(compute_ndvi_float(ref_b1[i], ref_b2[i]))
    
    return np.array(ndvi)

Smoothing functions

In [49]:
def ndvi_smoothing(y: np.ndarray, pvalue: float, nopval: bool, lrange: np.ndarray, nd: int):
    
     # create weights
    w = np.array((y!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z, lopt, vcurve, l = ws2doptv(y, w, array.array('d',lrange))
    else:
        z, lopt, vcurve, l = ws2doptvp(y, w, array.array('d',lrange), pvalue)
    
    return (z, lopt, vcurve, l)


def ref_smoothing(y1: np.ndarray, y2: np.ndarray, pvalue: float, nopval: bool, lrange: np.ndarray, nd: int):

     # create weights
    w1 = np.array((y1!=nd)*1,dtype='double')
    w2 = np.array((y2!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z1, lopt1, vcurve1, l1 = ws2doptv(y1, w1, array.array('d',lrange))
        z2, lopt2, vcurve2, l2 = ws2doptv(y2, w2, array.array('d',lrange))
    else:
        z1, lopt1, vcurve1, l1 = ws2doptvp(y1, w1, array.array('d',lrange), pvalue)
        z2, lopt2, vcurve2, l2 = ws2doptvp(y2, w2, array.array('d',lrange), pvalue)
    
    return (z1, z2, lopt1, lopt2, vcurve1, vcurve2, l1, l2)

Print and Plot functions

In [44]:
def print_info(location: str, latlon: dict, lagCorr1: float, lagCorr2: float):
    
    print('\033[1m' + 'Selected Point: ', location, '\033[0m')
    print('(Lat, Lon) =', latlon[location] )
    print('\n')
    
    print('LagCorr of NDVI: ', round(lagCorr1,3))
    print('LagCorr of Reflectance: ', round(lagCorr2,3))
    print('\n')
    
    
    
    
def plot_all(A0: bool, B1:bool, B2:bool,
             ndvi: np.ndarray, derived_ndvi: np.ndarray, 
             ndvi_z0: np.ndarray, ndvi_z1: np.ndarray, ndvi_z2: np.ndarray,
             ndvi_dts: np.ndarray, ref_c_dts: np.ndarray,
             nd: float, 
             yauto: bool, ylimits: tuple):
    
    
    #replace nd by nan
    ndvi_nan = ndvi.copy()
    ndvi_nan[ndvi_nan == nd] = np.nan
    
    derived_ndvi_nan = derived_ndvi.copy()
    derived_ndvi_nan[derived_ndvi_nan == nd] = np.nan
    
    plt.figure(figsize=(20,5))
    
    #raw values
    plt.plot(ndvi_dts, ndvi_nan, color = 'lightgrey', marker = 'o', alpha = 0.5)
    plt.plot(ref_c_dts, derived_ndvi_nan, color = 'grey', marker = 'o', alpha = 0.5)
    
    #smoothed values
    A = [A0, B1, B2]
    xA = [ndvi_dts, ref_c_dts, ref_c_dts]
    z = [ndvi_z0, ndvi_z1, ndvi_z2]
    col = ['b', 'r', 'g']
    label = ['A0', 'B1', 'B2']
    leg = ['raw ndvi', 'raw derived NDVI']
    
    for i,a in enumerate(A):
        if a:
            plt.plot(xA[i], z[i], color = col[i], alpha = 0.5)
            leg.append(label[i])

    
    if not(yauto):
        plt.ylim(ylimits)
    
    plt.xlabel('Date', fontsize=15)
    plt.ylabel('NDVI', fontsize=15)
    plt.legend(leg, fontsize=17, loc = 'lower right')
    plt.show()


    
def plot_vcurve(A0: bool, B1:bool, B2:bool,
                     ndvi_l0: np.ndarray, ndvi_l1: np.ndarray, ref_l1: np.ndarray, ref_l2: np.ndarray,
                     ndvi_vcurve0: np.ndarray, ndvi_vcurve1: np.ndarray, ref_vcurve1: np.ndarray, ref_vcurve2: np.ndarray, 
                     ndvi_lopt0: float, ndvi_lopt1: float, ref_lopt1: float, ref_lopt2: float):
    
    plt.figure(figsize=(20,10))
    
    
    A = [A0, B1, B2, B2]
    xA = [ndvi_l0, ndvi_l1, ref_l1, ref_l2]
    v = [ndvi_vcurve0, ndvi_vcurve1, ref_vcurve1, ref_vcurve2]
    lopt = [ndvi_lopt0, ndvi_lopt1, ref_lopt1, ref_lopt2]
    col = ['blue', 'red', 'green', 'darkgreen']
    band = ['','','(band 1) ', '(band 2) ']
    leg = []
    
    for i,a in enumerate(A):
        if a:
            plt.plot(xA[i], v[i], color = col[i], alpha = 0.5, marker = 'o')
            leg.append('lopt ' + band[i] + str(round(np.log10(lopt[i]),2)))
            
    for i,a in enumerate(A):
        if a:
            plt.axvline(x = np.log10(lopt[i]), ls = '--', color = col[i])

    plt.xlabel('log10(l)', fontsize=15)
    plt.ylabel('V', fontsize=15)
    plt.title('V-curves', fontsize=23)
    plt.legend(leg, fontsize=17, loc = 'lower right')
    plt.show()
            
    
def plot_year(A0: bool, B1:bool, B2:bool,
              year: int, month: int, 
              ndvi: np.ndarray, derived_ndvi: np.ndarray, 
              ndvi_z0: np.ndarray, ndvi_z1: np.ndarray, ndvi_z2: np.ndarray, 
              ndvi_dts: np.ndarray, ref_c_dts: np.ndarray, 
              nd: float, 
              yauto: bool, ylimits: tuple):

    
    #replace nd by nan
    ndvi_nan = ndvi.copy()
    ndvi_nan[ndvi_nan == nd] = np.nan
    
    derived_ndvi_nan = derived_ndvi.copy()
    derived_ndvi_nan[derived_ndvi_nan == nd] = np.nan
    
    
    plt.figure(figsize=(20,10))

    # Cropping time series to year
    year_index = np.all([ndvi_dts>=datetime.date(year,month,1), ndvi_dts<datetime.date(year+1,month,1)], axis=0)
    ndvi_dts = ndvi_dts[year_index]
    ndvi_nan = ndvi_nan[year_index]
    ndvi_z0 = np.array(ndvi_z0)[year_index]

    year_index2 = np.all([ref_c_dts>=datetime.date(year,month,1), ref_c_dts<datetime.date(year+1,month,1)], axis=0)
    ref_c_dts = ref_c_dts[year_index2]
    derived_ndvi_nan = derived_ndvi_nan[year_index2]
    ndvi_z1 = np.array(ndvi_z1)[year_index2]
    ndvi_z2 = np.array(ndvi_z2)[year_index2]
        
    
    
    #raw values
    plt.plot(ndvi_dts, ndvi_nan, color = 'lightgrey', marker = 'o', alpha = 0.5)
    plt.plot(ref_c_dts, derived_ndvi_nan, color = 'grey', marker = 'o', alpha = 0.5)
    
    
    A = [A0, B1, B2]
    xA = [ndvi_dts, ref_c_dts, ref_c_dts]
    z = [ndvi_z0, ndvi_z1, ndvi_z2]
    col = ['b', 'r', 'g']
    label = ['A0', 'B1', 'B2']
    leg = ['raw NDVI', 'raw derived NDVI']
    
    for i,a in enumerate(A):
        if a:
            plt.plot(xA[i], z[i], color = col[i], alpha = 0.5)
            leg.append(label[i])
    
    if not(yauto):
        plt.ylim(ylimits)

    plt.xlabel('Date', fontsize=15)
    plt.ylabel('NDVI', fontsize=15)
    plt.legend(leg, fontsize=15, loc = 'lower right')
    plt.title('Year ' + str(year), fontsize = 20)
    

Main function

In [46]:
def main(A0: bool, B1:bool, B2:bool,
              ndvi_MD: tuple, ref_MD:tuple,
              location: str, latlon: dict, satellite: str, 
              pvalue:float, nopval: bool, 
              lrange1: tuple, lrange2: tuple, step: float, 
              nd: int, 
              date_begin: datetime.date, date_end: datetime.date, 
              year: int, month: int,
              yauto: bool, ylimits: tuple):
    
    lrange1 = np.arange(lrange1[0],lrange1[1], step)
    lrange2 = np.arange(lrange2[0],lrange2[1], step)
    
    #extract time series
    ndvi_df, ref_df = select_satellite(satellite, ndvi_MD, ref_MD)
    (ndvi, ndvi_dts, ndvi_c_dts, ndvi_same_c) = ndvi_extract_ts(ndvi_df, location, date_begin, date_end) 
    (ref_b1, ref_b2, ref_dts, ref_c_dts) = ref_extract_ts(ref_df, location, date_begin, date_end)
    
    #Deriving NDVI from raw reflectance
    derived_ndvi = compute_ndvi_array(ref_b1, ref_b2)
    
    #smoothing
    (ndvi_z0, ndvi_lopt0, ndvi_vcurve0, ndvi_l0) = ndvi_smoothing(ndvi, pvalue, nopval, lrange1, nd)
    (ndvi_z1, ndvi_lopt1, ndvi_vcurve1, ndvi_l1) = ndvi_smoothing(derived_ndvi, pvalue, nopval, lrange1, nd)
    (ref_z1, ref_z2, ref_lopt1, ref_lopt2, ref_vcurve1, ref_vcurve2, ref_l1, ref_l2) = ref_smoothing(ref_b1, ref_b2, pvalue, nopval, lrange2, nd)
    
    #deriving NDVI from smoothed reflectance
    ndvi_z2 = compute_ndvi_array(ref_z1, ref_z2)

    #lagCorr
    lagCorr1 = lag1corr(np.array(ndvi[0:len(ndvi)-1]), np.array(ndvi[1:]), nd)
    lagCorr2 = lag1corr(np.array(derived_ndvi[0:len(derived_ndvi)-1]), np.array(derived_ndvi[1:]), nd)
    
    #Prints and plots
    print_info(location, latlon, lagCorr1, lagCorr2)
    plot_all(A0, B1, B2, ndvi, derived_ndvi, ndvi_z0, ndvi_z1, ndvi_z2, ndvi_dts, ref_c_dts, nd, yauto, ylimits)
    plot_year(A0, B1, B2, year, month, ndvi, derived_ndvi, ndvi_z0, ndvi_z1, ndvi_z2, ndvi_dts, ref_c_dts, nd, yauto, ylimits)
    plot_vcurve(A0, B1, B2, ndvi_l0, ndvi_l1, ref_l1, ref_l2, ndvi_vcurve0, ndvi_vcurve1, ref_vcurve1, ref_vcurve2, ndvi_lopt0, ndvi_lopt1, ref_lopt1, ref_lopt2)
    

## Loading the data

**NDVI**

In [10]:
ndvi_nd = -3000

In [11]:
# loading ndvi_MOD data from csv
ndvi_MOD = pd.read_csv(
    'data/MOD13A2-MOD13A2-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MOD13A2_006__1_km_16_days_NDVI', 
               'MOD13A2_006__1_km_16_days_composite_day_of_the_year'],
    dtype = {'MOD13A2_006__1_km_16_days_composite_day_of_the_year': int})


#renaming the columns
ndvi_MOD = ndvi_MOD.rename(columns={"MOD13A2_006__1_km_16_days_NDVI": "NDVI", 
                        "MOD13A2_006__1_km_16_days_composite_day_of_the_year": "Composite_date"})


# Convert string Date to datetime.date
ndvi_MOD['Date'] = ndvi_MOD['Date'].apply(fromstring)


# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ndvi_MOD)):
    
    d = ndvi_MOD['Date'][i]
    c = ndvi_MOD['Composite_date'][i]
    
    if c != -1:    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(False)
    elif (compo[i]==compo[i-1]):
        same_compo.append(True)
    else:
        same_compo.append(False)
            
ndvi_MOD['Composite_date'] = compo   
ndvi_MOD['Same_composite'] = same_compo 

In [12]:
# loading MYD data from csv
ndvi_MYD = pd.read_csv(
    'data/MYD13A2-MYD13A2-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MYD13A2_006__1_km_16_days_NDVI', 
               'MYD13A2_006__1_km_16_days_composite_day_of_the_year'],
    dtype = {'MYD13A2_006__1_km_16_days_composite_day_of_the_year': int})


#renaming the columns
ndvi_MYD = ndvi_MYD.rename(columns={"MYD13A2_006__1_km_16_days_NDVI": "NDVI", 
                        "MYD13A2_006__1_km_16_days_composite_day_of_the_year": "Composite_date"})


# Convert string Date to datetime.date
ndvi_MYD['Date'] = ndvi_MYD['Date'].apply(fromstring)


# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ndvi_MYD)):
    
    d = ndvi_MYD['Date'][i]
    c = ndvi_MYD['Composite_date'][i]
    
    if c != -1:    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(False)
    elif (compo[i]==compo[i-1]):
        same_compo.append(True)
    else:
        same_compo.append(False)
            
ndvi_MYD['Composite_date'] = compo   
ndvi_MYD['Same_composite'] = same_compo 

In [13]:
#creating MXD

#changing the index to numbers
number_index1 = pd.Index(range(0,2*len(ndvi_MYD),2))
number_index2 = pd.Index(range(1,2*len(ndvi_MYD)+1,2))

ndvi_MYD_nb = ndvi_MYD.set_index(number_index1)
ndvi_MYD_nb['ID'] = ndvi_MYD.index
ndvi_MOD_nb = ndvi_MOD.set_index(number_index2)
ndvi_MOD_nb['ID'] = ndvi_MOD.index


#concatenating and resetting ID as index
ndvi_MXD = pd.concat([ndvi_MYD_nb, ndvi_MOD_nb]).sort_index()
ndvi_MXD = ndvi_MXD.set_index('ID')

#re-run same_composite
same_compo = []
for i in range(len(ndvi_MXD)):
    if (i==0 or i==1):
        same_compo.append(False)
    elif (ndvi_MXD['Composite_date'][i]==ndvi_MXD['Composite_date'][i-1] or ndvi_MXD['Composite_date'][i]==ndvi_MXD['Composite_date'][i-2]):
        same_compo.append(True)
    else:
        same_compo.append(False)
              
ndvi_MXD['Same_composite'] = same_compo 

**Reflectance**

In [92]:
# loading ndvi_MOD data from csv
ref_MOD = pd.read_csv(
    'data/MOD09A1-MOD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MOD09A1_006_sur_refl_b01', 
               'MOD09A1_006_sur_refl_b02',
               'MOD09A1_006_sur_refl_day_of_year'],
     dtype = {'MOD09A1_006_sur_refl_day_of_year': int})


#renaming the columns
ref_MOD = ref_MOD.rename(columns={"MOD09A1_006_sur_refl_b01": "B1", 
                        "MOD09A1_006_sur_refl_b02": "B2",
                        'MOD09A1_006_sur_refl_day_of_year': 'Composite_date'})


# Convert string Date to datetime.date
ref_MOD['Date'] = ref_MOD['Date'].apply(fromstring)

#Cropping to right dates
a = (ref_MOD['Date']>datetime.date(2002,7,4)).values
ref_MOD = ref_MOD[a]

# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ref_MOD)):
    
    d = ref_MOD['Date'][i]
    c = ref_MOD['Composite_date'][i]
    
    if c != 65535:    #because nd is 65535
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(True)
    elif (compo[i]==compo[i-1]):
        same_compo.append(False)
    else:
        same_compo.append(True)
            
ref_MOD['Composite_date'] = compo   
ref_MOD = ref_MOD[same_compo]

In [88]:
# loading ndvi_MOD data from csv
ref_MYD = pd.read_csv(
    'data/MYD09A1-MYD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MYD09A1_006_sur_refl_b01', 
               'MYD09A1_006_sur_refl_b02',
               'MYD09A1_006_sur_refl_day_of_year'],
     dtype = {'MYD09A1_006_sur_refl_day_of_year': int})


#renaming the columns
ref_MYD = ref_MYD.rename(columns={"MYD09A1_006_sur_refl_b01": "B1", 
                        "MYD09A1_006_sur_refl_b02": "B2",
                        'MYD09A1_006_sur_refl_day_of_year': 'Composite_date'})


# Convert string Date to datetime.date
ref_MYD['Date'] = ref_MYD['Date'].apply(fromstring)

# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ref_MYD)):
    
    d = ref_MYD['Date'][i]
    c = ref_MYD['Composite_date'][i]
    
    if c != 65535:    #because nd is 65535    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(True)
    elif (compo[i]==compo[i-1]):
        same_compo.append(False)
    else:
        same_compo.append(True)
            
ref_MYD['Composite_date'] = compo   
ref_MYD = ref_MYD[same_compo]

In [129]:
# Creating MXD

ref_MXD = pd.DataFrame()

# Blend MOD and MYD as function of composite_date for each location
locations = sorted(list(set(ref_MYD.index)))
for l in locations:
    
    MOD = ref_MOD.loc[l]
    MOD = MOD.reset_index()
    MOD = MOD.set_index('Composite_date')
    
    MYD = ref_MYD.loc[l]
    MYD = MYD.reset_index()
    MYD = MYD.set_index('Composite_date')
    
    MXD = pd.concat([MOD, MYD]).sort_index()
    MXD = MXD.reset_index()
    MXD = MXD.set_index('ID')
    
    ref_MXD = pd.concat([ref_MXD, MXD])

# Remove values with same composite_date (keep max value)
same_compo = []
for i in range(len(ref_MXD)):    
    if (i==0):
        same_compo.append(True)
    elif (ref_MXD['Composite_date'][i]==ref_MXD['Composite_date'][i-1]):
        if compute_ndvi_float(ref_MXD['B1'][i-1],ref_MXD['B2'][i-1]) > compute_ndvi_float(ref_MXD['B1'][i],ref_MXD['B2'][i]): 
            same_compo.pop()
            same_compo.append(False)
            same_compo.append(True)
        else:
            same_compo.append(False)
    else:
        same_compo.append(True)
        
ref_MXD = ref_MXD[same_compo]

**ID lat lon**

In [10]:
#Creating lat lon dictionnary

latlon = {}

pd_latlon = pd.read_csv(
    'data/MYD13A2-MYD13A2-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Latitude', 
               'Longitude'])

dict_index = set(pd_latlon.index)

for location in dict_index:
    latlon[location] = (round(pd_latlon.loc[location]['Latitude'][0],3), round(pd_latlon.loc[location]['Longitude'][0],3))